In [ ]:
reset

In [13]:
# -*- coding: utf-8 -*-
#CNVD爬虫
import requests
from lxml import etree
import csv
import time
import random
from collections import OrderedDict
import codecs
from datetime import date
from multiprocessing.dummy import Pool as Threadpool
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.expected_conditions import alert_is_present
from selenium.webdriver.common.alert import Alert
import ast
import json
import re
import os
import string
import cookielib
import urllib2

# 判断是否爬取页面信息，为1则重新爬并缓存页面，为0则直接从缓存页面捕获信息。
is_crwal = 0

class Wenkuspider(object):
    def __init__(self):
        self.__file__ = r'C:/Users/wz/Desktop/lvmeng/paloalt_submit00.json'
        self.__link__ = r"https://security.paloaltonetworks.com"
        self.__loadeddir__ = 'loadedhtml0'
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}
        # 如果从某处断线了，可以更改起始的url地址
        #self.start_url = "http://www.cnvd.org.cn/flaw/list.htm"
        self.count = 0
        self.cookies = self.get_cookies()

    # 保存网页源码到本地，local_filename表示当前页面的完整名称
    def download_document(self, url, local_filename, retry=3):
#         print(url)
#         print(local_filename)
        local_dirname = os.path.dirname(local_filename)
        if not os.path.isdir(local_dirname):
            print "create dir:", local_dirname
            os.makedirs(local_dirname)
        req = urllib2.Request(url)
        req.add_header('User-Agent',
                       'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3135.5 Safari/537.36')
        retry_init = retry
        while retry:
            try:
                resp = urllib2.urlopen(req, timeout=30)
                content = resp.read()
                f = open(local_filename, "w")
                f.write(content)
                f.close()
                break
            except Exception, e:
                print e
                retry = retry - 1
                print "retry %s after %ss ..." % (url, retry_init - retry)
                sleep(retry_init - retry)
    
    # 获取网站cookie
    def get_cookies(self):
        # 构建一个CookieJar对象实例来保存cookie
        cookiejar = cookielib.CookieJar()
        # 使用HTTPCookieProcessor()来创建cookie处理器对象，参数为CookieJar()对象
        handler=urllib2.HTTPCookieProcessor(cookiejar)
        # 通过 build_opener() 来构建opener
        opener = urllib2.build_opener(handler)
        # 4. 以get方法访问页面，访问之后会自动保存cookie到cookiejar中
        opener.open(self.__link__)
        # 可以按标准格式将保存的Cookie打印出来
        cookieStr = ''
        for item in cookiejar:
            cookieStr += "'"+item.name + "':'" + item.value + "',"
        cookie = ast.literal_eval('{'+cookieStr+'}')
#         print(cookie)
        return cookie
    # get请求
    def parse(self, i):
#         time.sleep(random.randint(5, 10))
        self.count += 1
        print('每两个请求重新获取cookie，重置完成后，现在是第%s个'%str(self.count))
        if(self.count == 2):
            self.cookies = self.get_cookies()
            self.count = 0
        url=self.__link__+'/?page=%s&sort=date'%str(i)
        print(1)
        print(url)
        html = requests.get(url, headers=self.headers,
                            cookies=self.cookies).content
#         print(html)
        html = etree.HTML(html)
#         print('第%s页'%str(int(i/20+1)))
#         print(html)
        return html

    # get请求网页具体信息
    def parsedetail(self, url):
#         time.sleep(random.randint(5, 10))
        self.count += 1
        print('每两个请求重新获取cookie，重置完成后，现在是第%s个'%str(self.count))
        if(self.count == 2):
            self.cookies = self.get_cookies()
            self.count = 0
        html = requests.get(url, headers=self.headers,
                            cookies=self.cookies).content
#         print(html)
        html = etree.HTML(html)
#         print('第%s页'%str(int(i/20+1)))
#         print(html)
        return html

    # 对第i页网页列表逐个进行解析获取细节数据
    def get_list_url(self, html):
        # xpath具备较多缺陷，当在谷歌xpath插件上构建成功后，可以考虑删除tbody作为当前脚本的构建结果
        list_url = html.xpath('//*[@id="chartForm"]/div/div[2]/table//tr/td[2]/a/@href')  
#         print(list_url)
        for url0 in list_url:
            url = self.__link__ + url0
            self.parse_detail(url)

    # 解析缓存网页的详细信息
    def parse_loadedhtml(self, html,url):
        # item = OrderedDict()  # 如果要存入csv文档，建议用有序字典
        item = OrderedDict()
        # 获取cnvd id
        list_url=html.xpath('/html/head/title/text()')[0]
        # list_url.split()[0]
        if 'CVE' in list_url.split()[0]:
            item["app_name"] = 'Palo Alto Networks'
            item["cve_list"] = list_url.split()[0]
            
            # 获取漏洞影响的产品版本号
            list_version = html.xpath(
                '//table[@class="neat card"]//td/text()')
            # list_version
            # ''.join(list_version)
            versions={}
            if 'Windows' not in ''.join(list_version) and 'OS X' not in ''.join(list_version):
                for i in range(len(list_version)):
                    if '<' in list_version[i]:
                        versions[list_version[i-1]]=''.join(list(filter(lambda ch: ch in '0123456789.', list_version[i])))
                item["packages"] = versions
#                 print(item["packages"])
            
                # 获取sa_id
                item["sa_id"] = html.xpath(
                    "//h4/text()")
                if 'PAN-OS' in item["sa_id"][0]:
                    item["sa_id"] = "_".join((item["cve_list"],item["sa_id"][0]))

                    # 获得sa_link
                    item["sa_link"] = url

                    # 获得sa_title
                    item["sa_title"] = html.xpath('/html/head/title/text()')
                    item["sa_title"] = item["sa_title"][0]

                    # 获得sa_url
                    item["sa_url"] = url
                    
                    # 保存为json格式
                    item = [item]
                    # 输出：[OrderedDict([('target', 'total_result'), ('key1', 'value1'), ('key2', 'value2'), ('key3', 'value3')])]
                    current_dir = os.path.dirname(self.__file__)
                    path = os.path.join(os.path.join(current_dir), self.__file__.split('/')[-1])
#                     print(3)
#                     print(path)
#                     input()
                    with open(path, 'a') as f:
                        json.dump(item, f, encoding="utf-8", ensure_ascii=False, indent=4, separators=(',', ':'))
    
    # 获取详细漏洞信息数据，采用get请求方法
    def parse_detail(self, url):
#         time.sleep(random.randint(5, 10))
        html = self.parsedetail(url)
        # item = OrderedDict()  # 如果要存入csv文档，建议用有序字典
        item = OrderedDict()
        
        # 获取cnvd id
        list_url=html.xpath('/html/head/title/text()')[0]
        # list_url.split()[0]
        if 'CVE' in list_url.split()[0]:
            item["app_name"] = 'Palo Alto Networks'
            item["cve_list"] = list_url.split()[0]
            
            # 获取漏洞影响的产品版本号
            list_version = html.xpath(
                '//table[@class="neat card"]//td/text()')
            # list_version
            # ''.join(list_version)
            versions={}
            if 'Windows' not in ''.join(list_version) and 'OS X' not in ''.join(list_version):
                for i in range(len(list_version)):
                    if '<' in list_version[i]:
                        versions[list_version[i-1]]=''.join(list(filter(lambda ch: ch in '0123456789.', list_version[i])))
                item["packages"] = versions
#                 print(item["packages"])
            
                # 获取sa_id
                item["sa_id"] = html.xpath(
                    "//h4/text()")
                if 'PAN-OS' in item["sa_id"][0]:
                    item["sa_id"] = "_".join((item["cve_list"],item["sa_id"][0]))

                    # 获得sa_link
                    item["sa_link"] = url

                    # 获得sa_title
                    item["sa_title"] = html.xpath('/html/head/title/text()')
                    item["sa_title"] = item["sa_title"][0]

                    # 获得sa_url
                    item["sa_url"] = url
                    
                    # 保存为json格式
                    item = [item]
                    # 输出：[OrderedDict([('target', 'total_result'), ('key1', 'value1'), ('key2', 'value2'), ('key3', 'value3')])]
                    current_dir = os.path.dirname(self.__file__)
                    path = os.path.join(os.path.join(current_dir), self.__file__.split('/')[-1])
                                        
                    print(url)
                    local_filenames = os.path.join(os.path.dirname(self.__file__), self.__loadeddir__)
                    local_filenames = local_filenames+'/'+url.split('/')[-1]+'.html'
                    print(local_filenames) #C:/Users/wz/Desktop/lvmeng\loadedhtml
#                     input()
                    self.download_document(url, local_filenames,1)
                    
                    with open(path, 'a') as f:
                        json.dump(item, f, encoding="utf-8", ensure_ascii=False, indent=4, separators=(',', ':'))

    def run(self):        
        chrome_options = Options()
        # 加上下面两行，解决报错
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
#         chrome_options.add_argument("--headless")
        chrome_options.binary_location =  r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
        driver = webdriver.Chrome(chrome_options=chrome_options)
        #设置每次ChromeDriver访问的初始页面用来更新cookie，以绕过cnvd爬虫限制 OK
        driver.get("https://openapi.book118.com/login/login/index.html")
        raw_input()
        # 进入“我的文档”
        driver.find_element_by_xpath("//ul[@id='nav']/li[4]/a/span").click()
        time.sleep(3)
        # 进入“回收站”
        driver.find_element_by_xpath("//div[@id='broadside']/div/ul/li[8]/a/span/em").click()
        time.sleep(3)
        # 切换到frame框架
        driver.switch_to.frame(0)
        time.sleep(3)
        driver.find_element_by_xpath("//div[@id='batch']/div/div/label").click()
        time.sleep(2)
        driver.find_element_by_link_text(u"批量彻底删除").click()
        time.sleep(3)
        # 切换到默认窗口确认删除
        driver.switch_to_default_content()
        time.sleep(2)
        driver.find_element_by_xpath("//div[@class='layui-layer-btn layui-layer-btn-c']/a[1]").click()
        time.sleep(7)
        for i in range(1,300,1):
            time.sleep(3)
            # 切换到frame框架
            driver.switch_to.frame(0)
            time.sleep(3)
            driver.find_element_by_xpath("//div[@id='batch']/div/div/label").click()
            time.sleep(2)
            driver.find_element_by_link_text(u"批量彻底删除").click()
            time.sleep(3)
            # 切换到默认窗口确认删除
            driver.switch_to_default_content()
            time.sleep(2)
            driver.find_element_by_xpath("//div[@class='layui-layer-btn layui-layer-btn-c']/a[1]").click()
            time.sleep(7)
        raw_input()
        if is_crwal == 1:
            for i in range(1,10,1): 
                print('第%s页'%str(i))
                html = self.parse(i)
    #             print(i) #页数
                # 解析细节
                next_url = self.get_list_url(html)
        # 直接从本地缓存网页中捕获目标信息
        else:
            g = os.walk(os.path.join(os.path.dirname(self.__file__),self.__loadeddir__))
            for path,dir_list,file_list in g:  
                for file_name in file_list:  
                    file_name = os.path.join(path, file_name)
                    fp = open(file_name,'rb')
                    html_new = fp.read()
                    url = self.__link__+'/'+file_name.split('\\')[-1]
                    print(1)
                    print(url)
#                     input()
#                     print(html_new)
#                     import re
                    content=etree.HTML(html_new)
                    self.parse_loadedhtml(content,url)
#                     print(content)
            
        # 对所有数据进行json化
        f = open(self.__file__,'a+')
        s = f.read()
        s = s.replace('][',',')
        f.seek(0)
        f.truncate() #把原来的内容删掉
        f.write(s)
        f.close()

if __name__ == "__main__":
    a = Wenkuspider()
    pool = Threadpool(1)  # 单线程跑的慢，但是基本很稳定没有被cnvd封杀。本次跑完整个cnvd库大概十天左右，期间网络波动中断需要维护代码，及爬虫重新爬的页面范围。
    a.run()
    # pool.map(a.run(),self.parse())
    pool.close()
    pool.join()

c:\python27\lib\site-packages\ipykernel_launcher.py:245: DeprecationWarning: use options instead of chrome_options


c:\python27\lib\site-packages\ipykernel_launcher.py:263: DeprecationWarning: use driver.switch_to.default_content instead
c:\python27\lib\site-packages\ipykernel_launcher.py:277: DeprecationWarning: use driver.switch_to.default_content instead


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='batch']/div/div/label"}
  (Session info: chrome=83.0.4103.116)


In [4]:
print(1)
time.sleep(5)
print(2)

1
2
